In [1]:
"""The data stored in pickle format with version: 0.7.5 (python 3.7).
The type of graph object：networkx.classes.multidigraph.MultiDiGraph
Numbers of nodes: 2973489
Numbers of edges: 13551303
Average degree:   4.5574
Nodes' features：
    // The label. 1 means fishing mark node, otherwise 0.
    G.nodes[nodeName]['isp']；

Edges' features:
    G[node1][node2][0]['amount']        // The amount mount of the transaction.
    G[node1][node2][0]['timestamp']     // The timestamp of the transaction.				
							
* Notes * 
"""
# Quick start tutorial.

import pickle 
import networkx as nx

def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

G = load_pickle('/mnt/data/ethereum-phishing-transaction-network/MulDiGraph.pkl')

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7a41eb4ff040>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/gcloud/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [17]:
# Traversal nodes:
for idx, nd in enumerate(nx.nodes(G)):
    print(nd)
    print(G.nodes[nd]['isp'])
    break
# Travelsal edges:
c = 0
for ind, edge in enumerate(nx.edges(G)):
    (u, v) = edge
    eg = G[u][v][0]
    print(G[u][v])
    amo, tim = eg['amount'], eg['timestamp']
    print(u, v, amo, tim)
    eg = G[u][v][0]
    if c == 10:
        break
    else:
        c += 1

0x1f1e784a61a8ca0a90250bcd2170696655b28a21
0
{0: {'amount': 2.3446233, 'timestamp': 1526454086.0}}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 0x1266f8b9e4dffc9e2f719bf51713f7e714516861 2.3446233 1526454086.0
{0: {'amount': 0.07, 'timestamp': 1504461965.0}, 1: {'amount': 0.052111, 'timestamp': 1504473420.0}}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de 0.07 1504461965.0
{0: {'amount': 0.07, 'timestamp': 1504461965.0}, 1: {'amount': 0.052111, 'timestamp': 1504473420.0}}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 0x806ceb189d36700a97f4e7ecd4fb6c95f2c5a3de 0.07 1504461965.0
{0: {'amount': 5.06854256, 'timestamp': 1507806219.0}, 1: {'amount': 0.9925, 'timestamp': 1527097014.0}}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 0x3ec4688db6bf8464b0bef30ec2ca7afc7fa7863c 5.06854256 1507806219.0
{0: {'amount': 5.06854256, 'timestamp': 1507806219.0}, 1: {'amount': 0.9925, 'timestamp': 1527097014.0}}
0x1f1e784a61a8ca0a90250bcd2170696655b28a21 0x3ec4688db6bf8464b

In [32]:
import time
from datetime import datetime, timezone

min_time = 1e20
max_time = 0
from_address = []
to_address = []
time_stamps = []
amounts = []
uniq = set()
for ind, edge in enumerate(nx.edges(G)):
    (u, v) = edge
    # from_address.append(u)
    # to_address.append(v)
    eg = G[u][v][0]
    # print(u, v, eg)
    # break
    amo, tim = eg['amount'], eg['timestamp']
    amounts.append(amo)
    d = datetime.fromtimestamp(tim, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S%z")
    # time_stamps.append(d)
    uniq.add((u, v, amo, tim))
    min_time = min(min_time, tim)
    max_time = max(max_time, tim)
min_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(min_time))
max_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(max_time))
print(min_time, max_time)
print(len(uniq))

uniql = list(uniq)
from_address = [x[0] for x in uniql]
to_address = [x[1] for x in uniql]
amounts = [x[2] for x in uniql]
time_stamps = [datetime.fromtimestamp(x[3], tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S%z") for x in uniql]
print(len(from_address), len(to_address), len(time_stamps), len(amounts))

2015-08-07 07:01:09 2019-01-19 09:32:09
5355155
5355155 5355155 5355155 5355155


In [36]:
print(from_address[:10])
print(to_address[:10])
print(time_stamps[:10])
for f, t, ts, a in zip(from_address[:10], to_address[:10], time_stamps[:10], amounts[:10]):
    print(f, t, ts, a)

['0x0ca8786842fbabb8cf55fe9fb4bae05dadbd0bac', '0x3261b3e1e4da997e6c0493c0f5059f254d8cd831', '0x9432a11b969614d3852a90a9440959e231db25a2', '0x1f777ba20ea9ab436ee683280a0f2a21784c8a75', '0x422866a8f0b032c5cf1dfbdef31a20f4509562b0', '0xcdbf91f5447b17744273a8baae8d40d06b5d0a0d', '0xd42f82cfd43beb0811c79283df989777dde3220f', '0x88046d5e21618a7b385c2bf841154d66e68f7698', '0x717f660d514cd6004dd34b5661baf298b1f84805', '0x94234cfd63e20f280111daf8627c95c0ab2c3ac7']
['0xfc5772c279bb3a0bed4219c0de484deff3ed2241', '0x7b93aee4f78be3fc6fc9c46c086a776bd23c0dde', '0x5b2e4a700dfbc560061e957edec8f6eeeb74a320', '0x7bb29512edbb9074657a2e6fa756f94a6e1ba1e9', '0xd4455b712e19ba03d5a6261e32a7981aa268d641', '0xe120c1ecbfdfea7f0a8f0ee30063491e8c26fedf', '0x5d807e7f124ec2103a59c5249187f772c0b8d6b2', '0x1a7a8bd9106f2b8d977e08582dc7d24c723ab0db', '0x5275c72dcf681dac4006a987763a30b8918175ef', '0xb24754be79281553dc1adc160ddf5cd9b74361a4']
['2017-12-02 19:12:33+0000', '2018-01-12 22:40:45+0000', '2018-01-14 15:42:04+

In [2]:
from google.cloud import bigquery
client = bigquery.Client(project='cse3000')

In [41]:
# Perform a query.
QUERY = (
    # 'SELECT to_address, from_address FROM `bigquery-public-data.crypto_ethereum.transactions` '
    # f'WHERE to_address = {lol[0]}'
    # f'AND from_address = {lil[0]}'
    #f'WHERE to_address IN UNNEST({lol})'
    #f'AND from_address IN UNNEST({lil})'
    'SELECT nonce, transaction_index, from_address, to_address, value, gas, gas_price, input, receipt_gas_used, receipt_contract_address, receipt_status, block_timestamp, block_number, transaction_type, receipt_effective_gas_price  FROM `bigquery-public-data.crypto_ethereum.transactions`'
    #'SELECT COUNT(*) AS transaction_count FROM `bigquery-public-data.crypto_ethereum.transactions`'
    'WHERE block_timestamp BETWEEN TIMESTAMP("2015-08-07") AND TIMESTAMP("2019-01-20") AND from_address = "0xd4455b712e19ba03d5a6261e32a7981aa268d641" AND to_address = "0x422866a8f0b032c5cf1dfbdef31a20f4509562b0"'
    #'ORDER BY block_timestamp ASC LIMIT 10'

    # 'LIMIT 100'
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish
print(rows.num_results)

for row in rows:
    for x in row.keys():
        print(f"{x}: {row[x]}")
    break

0
nonce: 30
transaction_index: 70
from_address: 0xd4455b712e19ba03d5a6261e32a7981aa268d641
to_address: 0x422866a8f0b032c5cf1dfbdef31a20f4509562b0
value: 64100000000000000
gas: 100000
gas_price: 125000000000
input: 0xa6f2ae3a
receipt_gas_used: 72463
receipt_contract_address: None
receipt_status: 1
block_timestamp: 2018-01-10 04:33:21+00:00
block_number: 4883453
transaction_type: None
receipt_effective_gas_price: 125000000000


In [57]:

def get_transactions(from_address, to_address, time_stamps):
    assert len(from_address) == len(to_address) == len(time_stamps)
    # print(from_address[:10])
    # print(to_address[:10])
    # print(time_stamps[:10])

    conditions = " OR ".join(
        f'(from_address = "{from_addr}" AND to_address = "{to_addr}" AND block_timestamp = TIMESTAMP("{time_stamp}"))'
        for from_addr, to_addr, time_stamp in zip(from_address, to_address, time_stamps)
    )
    #print(conditions)

    query = (
        'SELECT `hash`, nonce, transaction_index, from_address, to_address, value, gas, gas_price, receipt_gas_used, receipt_contract_address, receipt_status, block_timestamp, block_number, max_fee_per_gas, max_priority_fee_per_gas, transaction_type, receipt_effective_gas_price FROM `bigquery-public-data.crypto_ethereum.transactions`'
        f'WHERE block_timestamp BETWEEN TIMESTAMP("2015-08-07") AND TIMESTAMP("2019-01-20") AND ({conditions})'
    )
    #print(query)

    query_job = client.query(query)  # API request
    rows = list(query_job.result())  # Waits for query to finish
    # print(len(rows))
    # for row in rows:
    #     print(row)
    #assert len(rows) == len(from_address)
    return rows
get_transactions(from_address[:30], to_address[:30], time_stamps[:30])

[Row(('0x1de814fc9744508adc707a6b232dfdea1f2eb4a0362e2e917177e902a24b7eb6', 3705, 80, '0x4c34ae54dc716808e94af3d1d638b8ea3a23fa9b', '0xf5b5cf8c446799e7afd6a0cd3f52a3f5e2475624', Decimal('1757259380000000000'), 63500, 20000000000, 21000, None, None, datetime.datetime(2017, 10, 8, 11, 22, 38, tzinfo=datetime.timezone.utc), 4347515, None, None, None, 20000000000), {'hash': 0, 'nonce': 1, 'transaction_index': 2, 'from_address': 3, 'to_address': 4, 'value': 5, 'gas': 6, 'gas_price': 7, 'receipt_gas_used': 8, 'receipt_contract_address': 9, 'receipt_status': 10, 'block_timestamp': 11, 'block_number': 12, 'max_fee_per_gas': 13, 'max_priority_fee_per_gas': 14, 'transaction_type': 15, 'receipt_effective_gas_price': 16}),
 Row(('0x9b246b74f34a4a33ec7caa5a2856ff1133b98fd8c139d7c372c873a6be58da73', 2046, 108, '0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be', '0x0b117c0eccb2e31fbc9eed994a125b2bde9c7e92', Decimal('396315000000000000'), 21001, 20000000000, 21000, None, None, datetime.datetime(2017, 8, 12,

In [61]:
# write rows to csv
import csv
transactions_path = '/mnt/data/ethereum-phishing-transaction-network/transactions.csv'
batch = 30
read = 0
start = 354660
with open(transactions_path, 'a') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'receipt_gas_used', 'receipt_contract_address', 'receipt_status', 'block_timestamp', 'block_number', 'max_fee_per_gas', 'max_priority_fee_per_gas', 'transaction_type', 'receipt_effective_gas_price'])
    writer.writeheader()
    for i in range(start, len(from_address), batch):
        print(i, i+batch)
        rows = get_transactions(from_address[i:i+batch], to_address[i:i+batch], time_stamps[i:i+batch])
        read += len(rows)
        print(read)
        for row in rows:
           writer.writerow(row)
        writer.writerow(row)

354660 354690
29
354690 354720
59
354720 354750
87
354750 354780
114
354780 354810
144
354810 354840
173
354840 354870
203
354870 354900
232
354900 354930
261
354930 354960
288
354960 354990
316
354990 355020
344
355020 355050
374
355050 355080
404
355080 355110
434
355110 355140
463
355140 355170
493
355170 355200
522
355200 355230
552
355230 355260
580
355260 355290
609
355290 355320
639
355320 355350
668
355350 355380
697
355380 355410
727
355410 355440
756
355440 355470
784
355470 355500
814
355500 355530
844
355530 355560
875
355560 355590
905
355590 355620
935
355620 355650
965
355650 355680
994
355680 355710
1022
355710 355740
1052
355740 355770
1081
355770 355800
1110
355800 355830
1140
355830 355860
1170
355860 355890
1198
355890 355920
1229
355920 355950
1258
355950 355980
1288
355980 356010
1318
356010 356040
1348
356040 356070
1378
356070 356100
1407
356100 356130
1437
356130 356160
1467
356160 356190
1497
356190 356220
1526
356220 356250
1555
356250 356280
1585
356280 3563

KeyboardInterrupt: 

In [2]:
import pandas as pd
# get raw transactions
df = pd.read_csv('/mnt/data/ethereum-phishing-transaction-network/transactions.csv')
# remove duplicates and save
df.drop_duplicates(inplace=True)
df.to_csv('/mnt/data/ethereum-phishing-transaction-network/transactions.csv', index=False)

In [ ]:
c = 0
def count_matching_edges(edge_list, df):
    # Convert the edge list to a set for faster lookup
    edge_set = set(edge_list)
    
    # Create a boolean mask for matching edges
    mask = df.apply(lambda row: (row['from_address'], row['to_address']) in edge_set, axis=1)
    
    
    # Count the number of True values in the mask
    count = mask.sum()
    
    return count

edge_list = list(G.edges())
matching_count = count_matching_edges(edge_list, df)
print(f"Number of matching edges: {matching_count}")
print(f"Number of total edges: {len(edge_list)}")
print(f"Percentage of matching edges: {matching_count/len(edge_list)}")
